### Step 1. 讀資料進來

In [ ]:
import pandas as pd
import re

In [ ]:
data = pd.read_csv("/content/spoiler_sentence_final.csv")
data.head()

,Unnamed: 0,Unnamed: 0.1,text,label,adverb,first_personal,reporting_verbs,ner,sentiment
0,0,0,最近閒來沒事，又把侏1翻出來看一遍，劇情、特效對我來說還是經典，很難被超越。,0.0,1,1,1,1,1
1,1,1,這部電影我從小到大也看了超過20次了吧，然而一些不合理處小時候沒感覺，這次看完後才覺得怪怪的,0.0,0,1,1,1,0
2,2,2,最後暴龍為什麼要救主角一行人？,1.0,0,0,0,0,1
3,3,3,明明一開始都是要致他們於死地的，怎麼最後反而成了救星？,1.0,0,0,0,0,1
4,4,4,暴龍是怎麼跑進遊客中心的？,1.0,0,0,0,0,1


### Step 2. 斷詞

In [ ]:
!pip install ckiptagger

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from ckiptagger import data_utils
data_utils.download_data_url("./")

In [ ]:
from ckiptagger import WS, POS, NER

ws = WS("./data")
pos = POS("./data")
ner = NER("./data")

/usr/local/lib/python3.8/dist-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/usr/local/lib/python3.8/dist-packages/ckiptagger/model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/usr/local/lib/python3.8/dist-packages/ckiptagger/model_ner.py:57: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)


In [ ]:
token_list = []
for text in data["text"]:
  tokens = ws([text])
  token_list.append(tokens)

In [ ]:
tokens = []
for i in token_list:
  token = " ".join(i[0])
  tokens.append(token)

In [ ]:
POS_list = []
for i in token_list:
  pos_tokens = pos(i)
  POS_list.append(pos_tokens)

In [ ]:
pos_tags = []
for i in POS_list:
  tags = " ".join(i[0])
  pos_tags.append(tags)

In [ ]:
NER_list = []
for i in range(len(token_list)):
  ner_token = ner(token_list[i], POS_list[i])
  NER_list.append(ner_token)

In [ ]:
data["tokens"] = tokens

In [ ]:
data["tags"] = pos_tags

### Step 3. 找特徵

In [ ]:
# sentiment
LIWC = pd.read_csv("/content/LIWC.csv", sep=',')

pos = [31]
neg = [32]
for num_31 in pos:
    pos_df = LIWC[LIWC.isin([num_31]).any(1)].reset_index(drop=True)
for num_32 in neg:
    neg_df = LIWC[LIWC.isin([num_32]).any(1)].reset_index(drop=True)

In [ ]:
posemo_list = []

for pos in pos_df['%']:
    posemo_list.append(pos)

In [ ]:
negemo_list = []

for neg in neg_df['%']:
    negemo_list.append(neg)

In [ ]:
positive = []
negative = []
neutral = []
for tokens in token_list:
  pos_counter = 0
  neg_counter = 0
  neu = 0
  for i in tokens[0]:
    if i in posemo_list:
      pos_counter = pos_counter+1
  for i in tokens[0]:
    if i in negemo_list:
      neg_counter = neg_counter+1
  if pos_counter == 0 and neg_counter == 0:
    neu = 1
  else:
    new = 0
  positive.append(pos_counter)
  negative.append(neg_counter)
  neutral.append(neu)

In [ ]:
data["positive"] = positive
data["negative"] = negative
data["neutral"] = neutral

In [ ]:
data.info()

In [ ]:
emo = []
for tokens in token_list:
  pos_count = 0
  neg_count = 0
  for i in tokens[0]:
    if i in posemo_list:
      pos_count = pos_count+1
    if i in negemo_list:
      neg_count = neg_count+1
  if pos_count == 0 and neg_count == 0:
    result = "neutral"
  elif pos_count-neg_count > 0:
    result = "positive"
  elif pos_count-neg_count < 0:
    result = "negative"
  emo.append(result)

In [ ]:
data["sentiment"] = emo

In [ ]:
score = []
for i in range(len(data["text"])):
  adverbs = re.findall(r"Df", data["tags"][i])
  if data["neutral"][i] == 1:
    scores = 0
  elif data["positive"][i] == data["negative"][i]:
    scores = 1*(1.5**len(adverbs))
  else:
    base = data["positive"][i]-data["negative"][i]
    scores = base*(1.5**len(adverbs))
  score.append(scores)

In [ ]:
data["senti_score"] = score

In [ ]:
# adverbs
adverb = []
for lists in POS_list:
  new_list = " ".join(lists[0])
  result = re.findall(r"Df", new_list)
  if len(result) > 0:
    adverb.append(1)
  else:
    adverb.append(0)

data["adverb"] = adverb

In [ ]:
# NER
ner_list = []
for lists in NER_list:
  result = len(lists[0])
  if result > 0:
    ner_list.append(1)
  else:
    ner_list.append(0)

data["ner"] = ner_list

In [ ]:
# first personal pronoun
first = []
for i in data["text"]:
  result = re.findall(r"我", i)
  if len(result) == 0:
    first.append(0)
  else:
    first.append(1)

data["first_personal"] = first

In [ ]:
# reporting verb
re_verbs = []
for verbs in data['text']:
  verbs = re.findall(r'說|講|提到|以為|提及|提出|問|道|表示|覺得|懷疑', verbs)
  if len(verbs) == 0:
    re_verbs.append(0)
  else:
    re_verbs.append(1)

data['reporting_verbs'] = re_verbs

In [ ]:
# aspect markers
aspect_markers = []
for aspect in data['tokens']:
  aspect = re.findall(r'\s(了|著|過|完|在|起來|正在|正|下去)\s', aspect)
  if len(aspect) == 0:
    aspect_markers.append(0)
  else:
    aspect_markers.append(1)

data['aspect_markers'] = aspect_markers

In [ ]:
data.to_csv(r"/content/features.csv", encoding = "UTF_8", index=False)